In [7]:
using System;
using System.Collections.Generic;
using System.Linq;

public enum OrderStatus
{
    Created,
    InProgress,
    Completed
}

public class Customer
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string PhoneNumber { get; set; }

    public Customer(string firstName, string lastName, string phoneNumber)
    {
        FirstName = firstName;
        LastName = lastName;
        PhoneNumber = phoneNumber;
    }
}

public class Employee
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Position { get; set; }
    public int OrdersProcessed { get; set; }

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
        OrdersProcessed = 0;
    }
}

public class Order
{
    public int OrderId { get; set; }
    public string Description { get; set; }
    public Employee AssignedEmployee { get; set; }
    public OrderStatus Status { get; set; }
    public DateTime CreatedAt { get; set; }
    public Customer Customer { get; set; }


    public Order(string description, Customer customer)
    {
        Description = description;
        Customer = customer;
        CreatedAt = DateTime.Now;
        Status = OrderStatus.Created;
    }
}

public class Company
{
    public string Name { get; set; }
    public List<Employee> Employees { get; set; }
    public List<Order> Orders { get; set; }

    public event Action<Order, OrderStatus> OrderStatusChanged;

    public Company(string name)
    {
        Name = name;
        Employees = new List<Employee>();
        Orders = new List<Order>();
    }

    public void AddEmployee(Employee employee)
    {
        Employees.Add(employee);
    }

    public int AddOrder(Order order)
    {
        order.OrderId = Orders.Count + 1; //Уникальный ID
        Orders.Add(order);
        return order.OrderId;
    }


    public void AssignEmployeeToOrder(int orderId, Employee employee)
    {
        var order = Orders.FirstOrDefault(o => o.OrderId == orderId);
        if (order != null)
        {
            order.AssignedEmployee = employee;
        }
    }

    public void ChangeOrderStatus(int orderId, OrderStatus newStatus)
    {
        var order = Orders.FirstOrDefault(o => o.OrderId == orderId);
        if (order != null)
        {
            OrderStatus oldStatus = order.Status;
            order.Status = newStatus;
            OrderStatusChanged?.Invoke(order, newStatus); // Вызов события
            if (newStatus == OrderStatus.Completed)
            {
                order.AssignedEmployee.OrdersProcessed++;
            }
        }
    }

    public List<Order> GetOrdersByStatus(OrderStatus status)
    {
        return Orders.Where(o => o.Status == status).ToList();
    }

    public List<Order> FindOrdersByCustomer(string customerFirstName, string customerLastName, string customerPhoneNumber)
    {
        return Orders.Where(o => o.Customer.FirstName == customerFirstName && o.Customer.LastName == customerLastName && o.Customer.PhoneNumber == customerPhoneNumber).ToList();
    }

    public void PrintEmployeeReport()
    {
        Console.WriteLine("Отчет по сотрудникам:");
        foreach (var employee in Employees)
        {
            Console.WriteLine($"{employee.FirstName} {employee.LastName} - Обработано заказов: {employee.OrdersProcessed}");
        }
    }
}

Company company = new Company("ExampleCompany");

// Добавление сотрудников
company.AddEmployee(new Employee("Софья", "Чепурных", "Менеджер"));
company.AddEmployee(new Employee("Аделия", "Шаукерова", "Оператор"));

//Добавление заказов
company.AddOrder(new Order("Заказ на доставку", new Customer("Валерия", "Шананина", "0001")));
company.AddOrder(new Order("Заказ на вывоз мебели", new Customer("Анастасия", "Косишнева", "0002")));

// Назначение сотрудника
company.AssignEmployeeToOrder(1, company.Employees[0]);
company.AssignEmployeeToOrder(2, company.Employees[1]);

//Обработка событий
Console.WriteLine("Обновление статусов заказов:");
company.OrderStatusChanged += (order, status) =>
{
    Console.WriteLine($"Статус заказа {order.OrderId} изменен на {status}");
    if (order.AssignedEmployee != null)
    {
        Console.WriteLine($"Ответственный: {order.AssignedEmployee.FirstName} {order.AssignedEmployee.LastName}");
    }
};

// Изменение статуса
company.ChangeOrderStatus(1, OrderStatus.InProgress);
company.ChangeOrderStatus(1, OrderStatus.Completed);
company.ChangeOrderStatus(2, OrderStatus.InProgress);

// Вывод заказов по статусу
Console.WriteLine();
Console.WriteLine("Заказы в процессе:");
foreach (var order in company.GetOrdersByStatus(OrderStatus.InProgress))
{
    Console.WriteLine($"ID: {order.OrderId}, Описание: {order.Description}");
}
Console.WriteLine();

Console.WriteLine("Завершенные заказы:");
foreach (var order in company.GetOrdersByStatus(OrderStatus.Completed))
{
    Console.WriteLine($"ID: {order.OrderId}, Описание: {order.Description}");
}
Console.WriteLine();

// Поиск заказов по клиенту
Console.WriteLine("Заказ клиента Валерия Шананина:");
foreach (var order in company.FindOrdersByCustomer("Валерия", "Шананина", "0001"))
{
    Console.WriteLine($"ID: {order.OrderId}, Описание: {order.Description}");
}
Console.WriteLine();

Console.WriteLine("Заказ клиента Анастасия Косишнева:");
foreach (var order in company.FindOrdersByCustomer("Анастасия", "Косишнева", "0002"))
{
    Console.WriteLine($"ID: {order.OrderId}, Описание: {order.Description}");
}
Console.WriteLine();

company.PrintEmployeeReport();

Обновление статусов заказов:
Статус заказа 1 изменен на InProgress
Ответственный: Софья Чепурных
Статус заказа 1 изменен на Completed
Ответственный: Софья Чепурных
Статус заказа 2 изменен на InProgress
Ответственный: Аделия Шаукерова

Заказы в процессе:
ID: 2, Описание: Заказ на вывоз мебели

Завершенные заказы:
ID: 1, Описание: Заказ на доставку

Заказ клиента Валерия Шананина:
ID: 1, Описание: Заказ на доставку

Заказ клиента Анастасия Косишнева:
ID: 2, Описание: Заказ на вывоз мебели

Отчет по сотрудникам:
Софья Чепурных - Обработано заказов: 1
Аделия Шаукерова - Обработано заказов: 0
